# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [67]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [68]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [69]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    scaler = None 
    pca = None
   
    if hyperparams['use_pca']:
        pca = PCA(n_components= hyperparams['pca_components'])  
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
            
    # Aggiunge il termine costante ai dati
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    X_val = np.c_[np.ones(X_val.shape[0]), X_val]
   
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        regolarizzatore = hyperparams['reg_lambda']
        I = np.eye(X_train.shape[1])       
        I[0, 0] = 0                              
        theta = np.linalg.inv(X_train.T @ X_train + regolarizzatore * I) @ X_train.T @ y_train
    else:
        theta = np.linalg.pinv(X_train) @ y_train

    # Calcolo predizioni
    y_train_pred = X_train @ theta
    y_val_pred = X_val @ theta

    # Calcola il MAE
    mae_train = np.mean(np.abs(y_train - y_train_pred))
    mae_val = np.mean(np.abs(y_val - y_val_pred))

    return mae_train, mae_val, theta, scaler, pca

# Dividi il dataset in training e test set
X, y = shuffle(X, y, random_state=42)

num_trainval = int(0.8 * len(X))

X_trainval = X[:num_trainval]
y_trainval = y[:num_trainval]

X_test = X[num_trainval:]
y_test = y[num_trainval:]
# non ho usato train_test_split(X, y, test_size=0.2, random_state=42) perchè non era stato importato
# ma avrei potuto usare anche quello

# Dividi il training set in training set effettivo e validation set

num_train = int(0.75 * len(X_trainval))
X_train = X_trainval[:num_train]
y_train = y_trainval[:num_train]
X_val = X_trainval[num_train:]
y_val = y_trainval[num_train:]

# Trova la configurazione di iperparametri migliore
best_config = None
best_val_mae = float('inf') #qualsiasi è migliore all'inizio
best_train_mae = None
best_weights = None

for config in configs:
    mae_train, mae_val, theta, scaler, pca = pipeline(X_train, y_train, X_val, y_val, config)
    if mae_val < best_val_mae:
        best_val_mae = mae_val
        best_train_mae = mae_train
        best_config = config
        best_theta = theta
        best_scaler = scaler
        best_pca = pca
# Riallena il modello sul training set completo
final_mae_train, final_mae_val, final_theta, _, _ = pipeline(X_training, y_training, X_test, y_test, best_config)

# Calcola il MAE sul test set
X_test = X_test.copy() #copio per non toccare i dati originari 

if best_config['data_standardize']:
    X_test = best_scaler.transform(X_test)  
    
if best_config['use_pca']:
    X_test = best_pca.transform(X_test)

X_test = np.c_[np.ones(X_test.shape[0]), X_test]

y_test_pred = X_test @ best_theta
mae_test =np.mean(np.abs(y_test - y_test_pred))

# Stampa  risultati
print("Best configuration:", best_config)
print("Best validation MAE:", round(best_val_mae, 4))
print("Best training MAE:", round(best_train_mae, 4))
print("Test set Mae:", round(mae_test, 4))

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Best configuration: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
Best validation MAE: 0.5833
Best training MAE: 0.5828
Test set Mae: 0.596
